In [131]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [132]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="leave_data"
    )

In [133]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None

In [134]:
#leave eligibility

def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"

In [135]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the dataset for leave classification
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    unwanted_keywords = ["feeling"]
    words = [word for word in words if word not in unwanted_keywords]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [136]:
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=3000)

In [137]:
# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [138]:
def check_casual_leave_exceeded(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        
        # Query to count the casual leave applications for the given email
        query = """
            SELECT empemail, COUNT(*) AS record_count
            FROM leaves
            WHERE leavetype = 'CASUAL LEAVE'
            AND DATE(applied) = DATE(`from`)
            AND MONTH(applied) = MONTH(CURRENT_DATE) 
            AND YEAR(applied) = YEAR(CURRENT_DATE)
            AND empemail = %s
            GROUP BY empemail
            HAVING COUNT(*) > 2;

        """
        
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()

        # Return whether the employee has exceeded casual leave applications more than 2 times
        if result:
            return True
        return False
    
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return False

In [139]:
# from datetime import timedelta
# import pandas as pd

# def get_weekdays(from_date, to_date):
#     # Ensure dates are in datetime format
#     from_date = pd.to_datetime(from_date)
#     to_date = pd.to_datetime(to_date)
    
#     # Generate dates for one day before 'from_date' and one day after 'to_date'
#     before_from_date = pd.date_range(start=from_date - timedelta(days=1), periods=1)
#     after_to_date = pd.date_range(start=to_date + timedelta(days=1), periods=1)
    
#     # Combine the two ranges into one DataFrame
#     date_range = pd.concat([pd.Series(before_from_date), pd.Series(after_to_date)])
    
#     # Create a DataFrame with dates and weekdays
#     weekdays_df = pd.DataFrame({
#         "Date": date_range,
#         "Weekday": date_range.dt.day_name()  # Get the weekday name
#     })
    
#     return weekdays_df


from datetime import date, timedelta
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the function to get weekdays
def get_weekdays(from_date, to_date):
    # Ensure dates are in datetime format
    from_date = pd.to_datetime(from_date)
    to_date = pd.to_datetime(to_date)
    
    # Adjust the range using timedelta
    date_range = pd.date_range(start=from_date - timedelta(days=1), end=to_date + timedelta(days=1))
    
    # Create a DataFrame with dates and weekdays
    weekdays_df = pd.DataFrame({
        "Date": date_range,
        "Weekday": date_range.day_name()  # Get the weekday name
    })
    
    return weekdays_df

# # Input dates
# from_date = "2024-12-09"
# to_date = "2024-12-14"

# # Get weekdays
# weekdays = get_weekdays(from_date, to_date)
# print(weekdays)


In [140]:
import mysql.connector
from datetime import datetime, timedelta

def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="leave_data"
    )

def high_leave_frequency(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        # Get current month and year for filtering
        current_month = datetime.now().month  # Hardcoded month, update as needed
        current_year = datetime.now().year
        
        # Query to fetch leaves for the provided email
        query = """
                SELECT 
                    `from`, 
                    `to`
                FROM leaves
                WHERE empemail = %s AND YEAR(`from`) = %s AND MONTH(`from`) = %s
                """
        cursor.execute(query, (email, current_year, current_month))
        leaves = cursor.fetchall()
        
        total_leave_days = 0

        # Loop through the fetched leave data
        for leave in leaves:
            from_date = leave['from']  # 'from' is the column name for the start date
            to_date = leave['to']      # 'to' is the column name for the end date

            # Calculate number of leave days between from_date and to_date
            leave_days = (to_date - from_date).days + 1  # Including the last day

            # Check if the day before the from_date is a Sunday
            if (from_date - timedelta(days=1)).weekday() == 6:
                leave_days += 1  # Add Sunday before leave

            # Check if the day after the to_date is a Sunday
            if (to_date + timedelta(days=1)).weekday() == 6:
                leave_days += 1  # Add Sunday after leave

            # Add this leave period's total days to the overall leave count
            total_leave_days += leave_days

        # Close the cursor and connection
        conn.close()

        # If the total leave days exceed 6, return True
        if total_leave_days > 6:
            return True

        return False

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return False


In [141]:
# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"

In [142]:
#short time leave application
def same_diff_leaves(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        
        # Corrected SQL query
        query = """
        SELECT 
            l.empemail
        FROM 
            leaves l
        WHERE 
            l.empemail = %s  -- Filter for specific employee email
            AND MONTH(l.from) = MONTH(CURRENT_DATE)  -- Current month
            AND YEAR(l.from) = YEAR(CURRENT_DATE)  -- Current year
            AND (DATEDIFF(l.to, l.from) = 0 OR DATEDIFF(l.to, l.from) = 1)  -- Leave duration of 0 or 1 day
        GROUP BY 
            l.empemail  -- Group by employee email
        HAVING 
            COUNT(*) > 2  -- Employees who applied for leave more than twice in the same month
        """
        
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()
        if result:
            return True
        return False
    
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return False
       


In [143]:
#final feature 2
from datetime import date, timedelta
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output



def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        from_date = from_date_input.value
        to_date = to_date_input.value
        selected_leave_type = leave_type_input.value
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)  # Assume model and vectorizer are available

        if not from_date or not to_date:
            print("Please select both 'From Date' and 'To Date'.")
            return

        elif from_date > to_date:
            print("'From Date' cannot be later than 'To Date'. Please correct the dates.")
            return
        same_diff = same_diff_leaves(email)
        l= "You are applying the leaves of same frequency" if same_diff else ""
        # if high_leave_frequency(email):
        #     print("Warning: Your leave frequency is high. You have already taken more than 6 days of leave this month.")
        check_casual_leave=check_casual_leave_exceeded(email)
        k="Leave Rejected: You have already applied for Casual Leave more than 2 times where applied == from dates" if check_casual_leave else ""
        # Check high leave frequency
        is_high_frequency = high_leave_frequency(email)
        h = "Warning: Your leave frequency is high. You have already taken more than 6 days of leave this month." if is_high_frequency else ""

        
        # Extend the date range and check for Sundays
        delta = (to_date - from_date).days + 1  # Number of days for the leave
        extended_weekdays = get_weekdays(from_date, to_date)
        sunday_count = extended_weekdays[extended_weekdays['Weekday'] == 'Sunday'].shape[0]

        # Adjust delta by adding the number of Sundays in the range
        delta += sunday_count

        # Only create the 'e' message when Sundays are present
        e = ""
        if sunday_count > 0:
            e = f"Sundays are included in the leave duration. {sunday_count} Sunday(s) were counted."

        if data is not None and not data.empty:
            # Check leave eligibility (whether user has enough balance)
            data = can_apply_leave(data)
            email = check_casual_leave_exceeded(email)
            # Check if the user is eligible to apply for leave
            if data["can_apply_leave"].iloc[0] == True:
                # Casual Leave case
                if selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    if leave_status != "Sick Leave" and today < from_date:
                        if delta <= data["lb"].iloc[0] and delta < 4:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Leave Granted")
                        elif delta > data["lb"].iloc[0]:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0:
                                print(e)  # Print only if Sundays exist
                            print("Requested leaves are exceeding the leave balance")
                            print(f"So it needs HR review & there will be LOP for {delta - data['lb'].iloc[0]} days")
                        elif delta > 3:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("As you are applying for more than 3 days, it needs HR review")
                    elif today == from_date:
                        # Check if the employee has exceeded casual leave more than 2 times
                        if selected_leave_type == "Casual Leave" and check_casual_leave:
                            print(k)
                        else:
                            if delta <= data["lb"].iloc[0] and delta < 4:
                                if is_high_frequency: print(h)
                                if same_diff:print(l)
                                if sunday_count > 0: print(e)  # Print only if Sundays exist
                                print("Leave Granted")
                            elif delta > data["lb"].iloc[0]:
                                if is_high_frequency: print(h)
                                if same_diff:print(l)
                                if sunday_count > 0: print(e)  # Print only if Sundays exist
                                print("Requested leaves are exceeding the leave balance")
                                print(f"So it needs HR review & there will be LOP for {delta - data['lb'].iloc[0]} days")
                            elif delta > 3:
                                if is_high_frequency: print(h)
                                if same_diff:print(l)
                                if sunday_count > 0: print(e)  # Print only if Sundays exist
                                print("As you are applying for more than 3 days, it needs HR review")
                    elif today > from_date:
                        print("Today's date should be less than the from date")
                
                # Sick Leave case
                elif selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    if today <= from_date:
                        if delta > data["lb"].iloc[0]:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Submit medical certificates after coming to office as you requested leaves are more than your leave balance")
                            print(f"LOP for {delta - data['lb'].iloc[0]} days")
                        elif delta <= data["lb"].iloc[0] and delta < 4:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Leave granted")
                        elif delta <= data["lb"].iloc[0] and delta >= 4:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Exceeding more than 3 days needs HR review. Submit the medical certificates after coming to office.")
                    elif today > from_date:
                        if today >= to_date:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Submit medical certificates")
                        else:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Submit medical certificates after coming")
                
                else:
                    print("Leave type isn't matching with the leave status")
            
            elif data["can_apply_leave"].iloc[0] == False:
                if selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    if leave_status != "Sick Leave" and today < from_date:
                        if delta >= 3:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Approving percentage is less")
                            print(f"So it needs HR review & there will be LOP for {delta} days")
                            print("You can't apply the leave, it can be applied through only HR")
                        elif delta < 3:
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            if same_diff:print(l)
                            print("You have chance of getting leave approved and it depends on the HR")
                            print(f"So it needs HR review & there will be LOP for {delta} days")
                            print("You can't apply the leave, it can be applied through only HR")
                    elif today >= from_date:
                        print("You cant apply on the same date as cls leave type ")
                
                elif selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    if today <= from_date and today < to_date:
                        if is_high_frequency: print(h)
                        if same_diff:print(l)
                        if sunday_count > 0: print(e)  # Print only if Sundays exist
                        print("Submit medical certificates after coming to office")
                        print(f"LOP for {delta} days because you don't have leave balance")
                        print("You can't apply the leave, it can be applied through only HR")
                    elif today > from_date:
                        if today >= to_date:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Submit medical certificates")
                            print(f"and your LOP will be {delta} days")
                        else:
                            if is_high_frequency: print(h)
                            if same_diff:print(l)
                            if sunday_count > 0: print(e)  # Print only if Sundays exist
                            print("Submit medical certificates after coming")
                            print("You can't apply the leave, it can be applied through only HR")
                
                else:
                    print("Leave type isn't matching with the leave status")
            
            else:
                print("You don't have enough leave balance")
        else:
            print("No data found for that email")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create date picker widgets for "From" and "To" inputs with min date set to today
today = date.today()
from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)

reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data:")
output = widgets.Output()

# Attach event listener to the button

submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([ 
    leave_type_input, 
    email_input, 
    from_date_input, 
    to_date_input, 
    reason_input, 
    submit_button, 
    output 
]))

c:\Users\it\anaconda3\Lib\site-packages\traitlets\traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(DatePicker).__init__(min=datetime.date(2024, 12, 14)).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(


In [144]:
def fetch_leave_data_for_previous_month():
    try:
        # Establish the database connection
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        
        query = """
        SELECT 
            l.empemail
        FROM 
            leaves l
        WHERE 
            MONTH(l.from) = MONTH(CURRENT_DATE)  -- Current month
            AND YEAR(l.from) = YEAR(CURRENT_DATE)  -- Current year
            AND (DATEDIFF(l.to, l.from) = 0 OR DATEDIFF(l.to, l.from) = 1)  -- Leave duration of 0 or 1 day
        GROUP BY 
            l.empemail  -- Group by employee email
        HAVING 
            COUNT(*) > 2  -- Employees who applied for leave more than twice in the same month
        """

        # Execute the query
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        
        # If results are found, convert them into a DataFrame
        if result:
            df = pd.DataFrame(result)
            return df
        else:
            # If no employees satisfy the criteria, return a message or an empty DataFrame
            print("No employees found with repeated leave requests.")
            return None
    except mysql.connector.Error as err:
        # Handle database connection errors
        print(f"Error: {err}")
        return None
leave_data = fetch_leave_data_for_previous_month()

# Display the result
if leave_data is not None:
    print(leave_data)
else:
    print("No data found.")

                   empemail
0  sowmya.nandipu@gmail.com


In [145]:
import voila

In [157]:
def check_casual_leave_exceeded(email):
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)
        
        # Query to count the casual leave applications for the given email
        query = """
        SELECT empemail, COUNT(*) AS record_count
FROM leaves
WHERE leavetype = 'CASUAL LEAVE'
  AND DATE(applied) = DATE(`from`)
  AND MONTH(applied) = MONTH(CURRENT_DATE) 
  AND YEAR(applied) = YEAR(CURRENT_DATE)
  AND empemail = %s
GROUP BY empemail
HAVING COUNT(*) > 2;

        """
        
        cursor.execute(query, (email,))
        result = cursor.fetchall()
        conn.close()

        # Return whether the employee has exceeded casual leave applications more than 2 times
        if result:
            print("done")
        return False
  
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return False
check_casual = check_casual_leave_exceeded(email="chanduwinner111@gmail.com")


done
